In [1]:
import numpy as np
import pandas as pd
from patsy import dmatrices
import warnings

In [2]:
def sigmoid(x):
    '''SIGMOID FUNCTION FOR X'''

    return 1/(1+np.exp(-x))

In [3]:
## Algorith settings

np.random.seed(0) # set the seed
tol=1e-8 # convergence tolerance
lam = None # l2 - regularization
max_iter = 20 # maximum allowed iterations 
r = 0.95 # covariance between x and z
n = 1000 # number of observations (size of dataset to generate)
sigma = 1 # variance of noise - how spread out is the data?

## Model settings

beta_x, beta_z, beta_v = -4, .9, 1 # true beta coefficients
var_x, var_z, var_v = 1, 1, 4 # variances of inputs

## the model specification i want to fit
formula = 'y ~ x + z + v + np.exp(x) + I(v**2 + z)'

In [4]:
# keeping x and z closely related (height and weight)
x, z = np.random.multivariate_normal([0,0], [[var_x, r], [r, var_z]], n).T
# blood pressure
v = np.random.normal(0, var_v, n)**3

# create a pandas dataframe
A = pd.DataFrame({'x': x, 'z': z, 'v': v})

# compute the log offs for our 3 independent variables
# using the sigmoid function
A['log_odds'] = sigmoid(A[['x', 'z', 'v']].dot([beta_x, beta_z, beta_v]) + sigma * np.random.normal(0,1,n))

#compute te probability sample from binomial distribuition
A['y'] = [np.random.binomial(1,p) for p in A.log_odds]

# create a dataframe that encompasses our input data, model formula, and outputs
y, X = dmatrices(formula, A, return_type = 'dataframe')

X.head()

/Users/FabianaBoldrin/PycharmProjects/hackerHank/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.


,Intercept,x,z,v,np.exp(x),I(v ** 2 + z)
0,1.0,-1.805133,-1.678592,-230.536312,0.164453,53145.312449
1,1.0,-1.320743,-0.612110,-321.120883,0.266937,103118.009125
2,1.0,-1.689545,-1.998587,0.006285,0.184604,-1.998547
3,1.0,-0.914205,-0.962069,-56.335960,0.400835,3172.778273
4,1.0,0.036999,0.166842,-0.033775,1.037692,0.167983


In [5]:
def catch_singularity(f):
    def silencer(*args, **kwargs):
        try:
            return f(*args, **kwargs)
        except np.linalg.LinAlgError:
            warnings.warn('Algotithm terminated - singular Hessian')
            return args[0]
        return silencer

In [ ]:
def newton_step(curr, X, lam=None):
    p = np.array(sigmoid(X.dot(curr[:,0])), ndmin=2).T
    w = np.diag((p*(1-p))[:,0])
    hessian = X.T.dot(W).dot(X)
    grad = X.T.dot(y-p)
    
    if lam:
        step, *_ = np.linalg.lstsq(hessian + lam*np.eye(curr.shape[0]), grad)
    else:
        step, *_ = np.linalg.lstsq(hessian, grad)
    
    beta = curr + step
    
    return beta